# New Relic ML Performance Monitoring - Bring Your Own Data

[ml-performance-monitoring](https://github.com/newrelic-experimental/ml-performance-monitoring) provides a Python library for sending machine learning models' inference data and performance metrics into New Relic.
<br>
By using this package, you can easily and quickly monitor your model, directly from a Jupyter notebook or a cloud service.
<br>
The package is ML framework agnostic and can be quickly integrated. It is based on the newrelic-telemetry-sdk-python library.
<br>
It is based on the [newrelic-telemetry-sdk-python](https://github.com/newrelic/newrelic-telemetry-sdk-python) library.


This notebook provides an example of sending inference data and metrics of an XGBoost model

<U>Note</U>- this notebook uses the libraries:
* numpy
* pandas
* sklearn
* tensorflow

### 0. Install libraries

In [ ]:
!pip3 install git+https://github.com/newrelic-experimental/ml-performance-monitoring.git

In [ ]:
!pip3 install pandas scikit-learn tensorflow

### 1. Import libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error

from ml_performance_monitoring.monitor import MLPerformanceMonitoring

### 2. Load the California housing prices dataset and split it into train and test sets

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

california_dataset = fetch_california_housing()
X, y = (
    california_dataset["data"],
    california_dataset["target"],
)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)

In [ ]:
print(X_train[:5], y_train[:5])

### 3. Fitting a Neural Network Regression Model with TensorFlow



In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='mse')

model.fit(X_train, y_train, epochs=10, batch_size=32)

### 4. Predicting the test set results

In [ ]:
y_pred = model.predict(X_test)
y_pred

### 5. Record inference data to New Relic

The MLPerformanceMonitoring parameters:
   * Required parameters:
      * `model_name` - must be unique per model
      *  `insert_key` - [Get your key](https://one.newrelic.com/launcher/api-keys-ui.api-keys-launcher) (also referenced as `ingest - license`) and set it as environment variable: `NEW_RELIC_LICENSE_KEY`.
[Click here](https://docs.newrelic.com/docs/apis/intro-apis/new-relic-api-keys/#license-key) for more details and instructions.

* Optional parameters:
   * `metadata` (dictionary) - will be added to each event (row) of the data
   * `send_data_metrics` (boolean) - send data metrics (statistics) to New Relic (False as default)
   * `features_columns`(list) - the features' names ordered as X columns.
   * `labels_columns` (list) - the labels' names ordered as y columns.

(note: The parameters `features_columns` and `labels_columns` are only relevant when sending the data as an np.array. When the data is sent as a dataframe, the dataframes (X,y) columns' names will be taken as features and labels names respectively. In addition, if you send your data as an np.array without sending the features_columns and labels_columns, on New Relic data, the names will appear as "feature_{n}" and "lablel_{n}" numbered by the features/labels order)


5.1. Define monitoring parameters

In [ ]:
metadata = {"environment": "notebook", "dataset": "California housing prices"}
model_version = "1.0"
features_columns, labels_columns = (
    list(california_dataset["feature_names"]),
    ["target"],
)

In [ ]:
import getpass
print("You can get your License-key here: https://one.newrelic.com/launcher/api-keys-ui.api-keys-launcher")
insert_key = getpass.getpass("Paste an License-key and hit enter:")

5.2 Create model monitor

In [ ]:
ml_monitor = MLPerformanceMonitoring(
    insert_key=insert_key,  # set the environment variable NEW_RELIC_LICENSE_KEY or send your license-key here
    model_name="TensorFlow Regression on California housing Dataset",
    metadata=metadata,
    send_data_metrics=True,
    features_columns=features_columns,
    labels_columns=labels_columns,
    model_version=model_version,
)

5.3 Send your data as an np.array.

In [ ]:
ml_monitor.record_inference_data(X=X_test, y=y_pred)

5.4  Send your data as a pd.DataFrame. The values have been transformed by the StandardScaler into z-scores.

In [ ]:
X_df = pd.DataFrame(
    X_test,
    columns=features_columns,
)

y_pred_df = pd.DataFrame(
    y_pred,
    columns=labels_columns,
)
X_df.head()

In [ ]:
ml_monitor.record_inference_data(X=X_df, y=y_pred_df)

In [ ]:
y_pred_df.head()

### 6. Record metrics to New Relic
You can stream custom metrics to New Relic, monitoring your model performance or model data. These metrics will be sent to NRDB as [metric data](https://docs.newrelic.com/docs/data-apis/ingest-apis/metric-api/introduction-metric-api/).

In [ ]:
rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)
print(f"RMSE: {rmse}")

In [ ]:
metrics = {
    "RMSE": rmse,
}
ml_monitor.record_metrics(metrics=metrics)

### 7. Monitor and alert
Done! Check your application in the [New Relic UI](https://one.newrelic.com/nr1-core?filters=%28domain%20%3D%20%27MLOPS%27%20AND%20type%20%3D%20%27MACHINE_LEARNING_MODEL%27%29) to see the real time data.